In [1]:
%pip install requests

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
import re
import pandas as pd
import requests


In [9]:


def get_coordinates_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            location = result["documents"][0]["address"]
            return float(location["y"]), float(location["x"])  # 위도, 경도 반환
        else:
            print(f"해당 주소에 대한 좌표 정보를 찾을 수 없습니다. {address}")
            return None, None
    else:
        print( f"에러 발생: {response.status_code}")
        return None, None


In [ ]:

#병원 점수 예측 데이터
trainedHospital = pd.read_csv('../export/hospital/2021_2024_09_trainDatas.csv')


block = 0
le = 0
# 사용 예제


for i in range(len(trainedHospital)):
    address = trainedHospital.iloc[i]['주소']
    address = address.split(' ')
    address = ' '.join(address[:4])
    coord = get_coordinates_kakao(address,'your key')
    trainedHospital.at[i,'lat'] = coord[0]
    trainedHospital.at[i,'lon'] = coord[1]

    block += 1
    if block > 100:
        print("--- " ,block , "  : ",le)
        block = 0
        le += 1

해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99
---  101   :  0
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 강남구 논현로 507-507
---  101   :  1
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 도봉구 삼양로 606-606
---  101   :  2
---  101   :  3
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99
---  101   :  4
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306
---  101   :  5
---  101   :  6
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99,
---  101   :  7
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  8
---  101   :  9
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 안암로 99-99,
---  101   :  10
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  11
---  101   :  12
---  101   :  13
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 서울특별시 성북구 동소문로 306-306,
---  101   :  14
---  101   :  15
---  101   :  16
---  101   :  17


In [11]:
trainedHospital.to_csv('../export/hospital/2021_2024_09_trainDatas.csv', index=False, encoding='utf-8') 